In [1]:
!pip install tensorflow keras opencv-python matplotlib numpy flask pillow


INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 621.2 kB/s eta 0:01:02
    --------------------------------------- 0.5/39.0 MB 621.2 kB/s eta 0:01:02
    --------------------------------------- 0.5/39.0 MB 621.2 kB/s eta 0:01:02
    --------------------------------------- 0.8/39.0 MB 508.5 kB/s eta 0:01:16
    --------------------------------------- 0.8/39.0 MB 508.5 kB/s eta 0:01:16
    -----------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.0 which is incompatible.


In [1]:
!pip install tensorflow==2.13


  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.2.0-py3-none-any.whl.metadata (6.4 kB)
INFO: pip is looking at multiple versions of grpcio to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
exceptiongroup 1.3.0 requires typing-extensions>=4.6.0; python_version < "3.13", but you have typing-extensions 4.5.0 which is incompatible.
ipython 8.37.0 requires typing_extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

train_dir = "dataset/train"
valid_dir = "dataset/valid"
test_dir  = "dataset/test"

# Image Generators
train_gen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

valid_gen = ImageDataGenerator(rescale=1/255)

train_data = train_gen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)

valid_data = valid_gen.flow_from_directory(
    valid_dir, target_size=(224, 224), batch_size=32, class_mode='binary'
)

# Pretrained Base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint("model/pneumonia_mobilenet.h5", save_best_only=True)

history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=5,
    callbacks=[checkpoint]
)

model.save("model/final_model.h5")


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
9406464/9406464 [==============================] - 6s 1us/step
Epoch 1/5
163/163 [==============================] - ETA: 0s - loss: 0.2821 - accuracy: 0.8785

c:\Users\anshu\anaconda3\envs\tf_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


163/163 [==============================] - 540s 3s/step - loss: 0.2821 - accuracy: 0.8785 - val_loss: 0.3453 - val_accuracy: 0.9375
Epoch 2/5
163/163 [==============================] - 410s 3s/step - loss: 0.1760 - accuracy: 0.9294 - val_loss: 0.3582 - val_accuracy: 0.8125
Epoch 3/5
163/163 [==============================] - 326s 2s/step - loss: 0.1465 - accuracy: 0.9433 - val_loss: 0.3852 - val_accuracy: 0.8125
Epoch 4/5
163/163 [==============================] - 319s 2s/step - loss: 0.1423 - accuracy: 0.9433 - val_loss: 0.4112 - val_accuracy: 0.8125
Epoch 5/5
163/163 [==============================] - 290s 2s/step - loss: 0.1224 - accuracy: 0.9538 - val_loss: 0.4016 - val_accuracy: 0.8125
